In [1]:
import sys
import warnings
sys.path.append("../Pythonscripts")
sys.path.append(r"C:\Users\atedi\OneDrive\Documents\SageBionetworks\synapse\Scripts")
import pandas as pd
import numpy as np
import synapseclient as sc
warnings.simplefilter("ignore")

In [2]:
## login to synapse
syn = sc.login()

Welcome, aryton tediarjo!



INFO:synapseclient_default:Welcome, aryton tediarjo!



In [113]:
## demographic data ## 
demographic_mpV2_data = ((syn.tableQuery("SELECT distinct(healthCode) as healthCode, \
                                         diagnosis as PD from syn15673379")
                    .asDataFrame()) \
                    .drop_duplicates("healthCode", keep = "first") \
                    .reset_index(drop = True)) 
demographic_mpV2_data = demographic_mpV2_data[demographic_mpV2_data["PD"] != "no_answer"] 
demographic_mpV2_data["PD"] = demographic_mpV2_data["PD"].map({"parkinsons":1, 
                                                               "control":0})

## get pdkit data ##
pdkit_mpV2_entity = syn.get("syn21018127")
pdkit_mpV2_data = pd.read_csv(pdkit_mpV2_entity["path"], index_col = 0)
pdkit_mpV2_data = (pdkit_mpV2_data[pdkit_mpV2_data["phoneInfo"].str.contains("iPhone")]) \
                        [(pdkit_mpV2_data != "#ERROR").all(axis = 1)] 
pdkit_mpV2_data = pdkit_mpV2_data[pdkit_mpV2_data["walk_motion.json_features_AA.no_of_steps"] \
                                  .astype(float) > 0]
pdkit_mpV2_data[[_ for _ in pdkit_mpV2_data.columns if "feat" in _]] = \
pdkit_mpV2_data[[_ for _ in pdkit_mpV2_data.columns if "feat" in _]].apply(pd.to_numeric)
pdkit_mpV2_data.reset_index(drop = True, inplace = True)
pdkit_mpV2_data = pd.merge(pdkit_mpV2_data, demographic_mpV2_data, 
                           how = "inner", on = "healthCode")

## get sfm data ## 
sfm_mpV2_entity = syn.get("syn21018245")
sfm_mpV2_data = pd.read_csv(sfm_mpV2_entity["path"], index_col = 0)
sfm_mpV2_data = (sfm_mpV2_data[sfm_mpV2_data["phoneInfo"].str.contains("iPhone")]) \
                        [(sfm_mpV2_data != "#ERROR").all(axis = 1)] 
sfm_mpV2_data[[_ for _ in sfm_mpV2_data.columns if "sfm" in _]] = \
sfm_mpV2_data[[_ for _ in sfm_mpV2_data.columns if "sfm" in _]].apply(pd.to_numeric)
sfm_mpV2_data.reset_index(drop = True, inplace = True)            
sfm_mpV2_data = pd.merge(sfm_mpV2_data, demographic_mpV2_data, 
                         how = "inner", on = "healthCode")